In [196]:
import numpy as np
import pandas as pd
import pennylane as qml
from susy_qm import calculate_Hamiltonian
from qiskit.quantum_info import SparsePauliOp
import itertools

In [153]:
potential = 'DW'
cutoff = 16

In [154]:
#calculate Hamiltonian and expected eigenvalues
H = calculate_Hamiltonian(cutoff, potential)

eigenvalues, eigenvectors = np.linalg.eig(H)
min_index = np.argmin(eigenvalues)
min_eigenvalue = eigenvalues[min_index]
min_eigenvector = np.asarray(eigenvectors[:, min_index])

min_3_ev = eigenvalues.argsort()[:3]

hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits

In [155]:
num_qubits

5

In [156]:
eig_vec = np.asarray(eigenvectors[:, min_3_ev[0]])
mat = eig_vec.reshape(2,cutoff)
mat = eig_vec.reshape([2]*num_qubits)
#mat = eig_vec.reshape(2,12)
#mat = eig_vec.reshape(8,4)

U, s, Vh = np.linalg.svd(mat, full_matrices=False)
p = s**2

entropy = -np.sum(p[p > 0] * np.log2(p[p > 0]))
print("Entanglement entropy:", entropy)

Entanglement entropy: 0.22875453169663273


In [200]:
potential = 'AHO'
cutoffs = [8, 16, 32]

for cutoff in cutoffs:

    print(f"Running for cutoff: {cutoff}")

    #calculate Hamiltonian and expected eigenvalues
    H = calculate_Hamiltonian(cutoff, potential)

    eigenvalues, eigenvectors = np.linalg.eig(H)
    min_index = np.argmin(eigenvalues)
    min_eigenvalue = eigenvalues[min_index]
    min_eigenvector = np.asarray(eigenvectors[:, min_index])

    hamiltonian = SparsePauliOp.from_operator(H)
    num_qubits = hamiltonian.num_qubits

    eig_vec = np.asarray(eigenvectors[:, min_3_ev[0]])
    psi_full = eig_vec.reshape(2, cutoff)
    psi_b = psi_full[0]

    psi_b_tensor = psi_b.reshape([2]*(num_qubits-1))

    n = psi_b_tensor.ndim
    positions = range(n)
    result = []

    for pos in itertools.combinations(positions, 1):
        perm = ['a'] * n
        perm[pos[0]] = 0
        result.append(perm)

    for i in range(len(result)):
        index_tuple = tuple()
        for idx in result[i]:
            if idx == 0:
                index_tuple += (0,)

            else:
                index_tuple += (slice(None),)

        psi_b1b2 = psi_b_tensor[index_tuple]

        U, s, Vh = np.linalg.svd(psi_b1b2, full_matrices=False)
        p = s**2

        # Compute the von Neumann entropy (entanglement entropy) for the bipartition between b1 and b2.
        entropy = -np.sum(p[p > 0] * np.log2(p[p > 0]))
        print(f"Entanglement entropy for {index_tuple}:", entropy)


Running for cutoff: 8
Entanglement entropy for (0, slice(None, None, None), slice(None, None, None)): -0.0
Entanglement entropy for (slice(None, None, None), 0, slice(None, None, None)): -0.0
Entanglement entropy for (slice(None, None, None), slice(None, None, None), 0): -0.0
Running for cutoff: 16
Entanglement entropy for (0, slice(None, None, None), slice(None, None, None), slice(None, None, None)): 0.1200287221113785
Entanglement entropy for (slice(None, None, None), 0, slice(None, None, None), slice(None, None, None)): 0.02464488830480452
Entanglement entropy for (slice(None, None, None), slice(None, None, None), 0, slice(None, None, None)): 0.14208918956879632
Entanglement entropy for (slice(None, None, None), slice(None, None, None), slice(None, None, None), 0): 0.0016293432855661997
Running for cutoff: 32
Entanglement entropy for (0, slice(None, None, None), slice(None, None, None), slice(None, None, None), slice(None, None, None)): 0.6311436026286741
Entanglement entropy for (s

In [187]:
eig_vec = np.asarray(eigenvectors[:, min_3_ev[0]])
psi_full = eig_vec.reshape(2, cutoff)
#psi_full
psi_b = psi_full[0]
#psi_b

psi_b_tensor = psi_b.reshape([2]*(num_qubits-1))
#psi_b_tensor

#psi_b1b2 = psi_b_tensor[:, 0, :, :]
index_tuple = (0,) + (slice(None),) * (psi_b_tensor.ndim - 1)
psi_b1b2 = psi_b_tensor[index_tuple]

#psi_b1b2

U, s, Vh = np.linalg.svd(psi_b1b2, full_matrices=False)
p = s**2

# Compute the von Neumann entropy (entanglement entropy) for the bipartition between b1 and b2.
entropy = -np.sum(p[p > 0] * np.log2(p[p > 0]))
print("Entanglement entropy between bosonic qubits b1 and b2:", entropy)


Entanglement entropy between bosonic qubits b1 and b2: 0.22685705444233492


In [160]:
eig_vec = np.asarray(eigenvectors[:, min_3_ev[0]])
psi_full = eig_vec.reshape(2, cutoff)
#psi_full
psi_b = psi_full[0]
#psi_b

psi_b_tensor = psi_b.reshape([2]*(num_qubits-1))
#psi_b_tensor

psi_b1b2 = psi_b_tensor[:, :, 0, :]
#psi_b1b2

U, s, Vh = np.linalg.svd(psi_b1b2, full_matrices=False)
p = s**2

# Compute the von Neumann entropy (entanglement entropy) for the bipartition between b1 and b2.
entropy = -np.sum(p[p > 0] * np.log2(p[p > 0]))
print("Entanglement entropy between bosonic qubits b1 and b2:", entropy)


Entanglement entropy between bosonic qubits b1 and b2: 0.052561976971423154


In [161]:
eig_vec = np.asarray(eigenvectors[:, min_3_ev[0]])
psi_full = eig_vec.reshape(2, cutoff)
#psi_full
psi_b = psi_full[0]
#psi_b

psi_b_tensor = psi_b.reshape([2]*(num_qubits-1))
#psi_b_tensor

psi_b1b2 = psi_b_tensor[:, :, :, 0]
#psi_b1b2

U, s, Vh = np.linalg.svd(psi_b1b2, full_matrices=False)
p = s**2

# Compute the von Neumann entropy (entanglement entropy) for the bipartition between b1 and b2.
entropy = -np.sum(p[p > 0] * np.log2(p[p > 0]))
print("Entanglement entropy between bosonic qubits b1 and b2:", entropy)


Entanglement entropy between bosonic qubits b1 and b2: 0.4134903922073404
